# Non linear analysis - Assignment 4

18.10.2023 - Jérémie Engler and Laure Toullier

In [6]:
#imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('bmh')
import math
import pandas as pd

np.set_printoptions(precision=5, suppress=True, linewidth=150)

Params = plt.rcParams
Params['figure.figsize'] = (14, 7) 

General data

In [7]:
E = 200e3 #Mpa
poisson = 0
sigma_y = 400 #Mpa

eps_0 = np.zeros((2,))
sigma_0 = np.zeros((2,))
sigma_y=400 #yield stress in MPa

strain_incr = np.array([0.003,0.004])

#Constitutive matrix for plane stress:
d = E/(1 - poisson**2) #Factor of the constitutive matrix

D = np.full((2,2), poisson)
np.fill_diagonal(D, 1)
D = D * d


Von Mises yield criterion

In [21]:
def f (vector, sigma_y):
    f=vector[0]**2+vector[1]**2-vector[0]*vector[1]-sigma_y
    return f

Plane stress condition

In [8]:
def stress_incr_computation(strain_incr, plastic_strain_incr, n, D):
    if plastic_strain_incr.all() == 0:
        stress_incr = D @ strain_incr
    else:
        stress_incr = (D - (D @ n.T @ n @ D) / (n @ D @ n.T)) @ strain_incr
        correction_factor = (D @ n.T @ n @ D) / (n @ D @ n.T)
        trial_factor = D

    return stress_incr

#Von Mises yield criterion
def check_failure(stress):
    fail = False
    criterion = stress[0]**2 + stress[1]**2 - stress[0] * stress[1] - sigma_y ** 2

    if criterion > 0:
        fail = True

    return fail

def strain_decomposition(strain_incr, alpha):
    elastic_strain_incr = alpha * strain_incr
    plastic_strain_incr = (1 - alpha) * strain_incr

    return elastic_strain_incr, plastic_strain_incr

def norm_criterion_surface(stress_y): #Unfinished
    n = np.array([[1,1]])
    return n

Euler backward method

In [14]:
#f derived by sigma

def d_f_d_sigma (f, sigma):
    d_f_d_sigma=np.zeros((2))
    d_f_d_sigma[0]=2*sigma[0]-sigma[1]
    d_f_d_sigma[1]=2*sigma[1]-sigma[0]
    return d_f_d_sigma

In [17]:
def f_of_incr_lambda(incr_lambda):
    f_of_incr_lambda=incr_lambda[0]**2+incr_lambda[1]**2-incr_lambda[0]*incr_lambda[1]-sigma_y
    return f_of_incr_lambda

In [15]:
#sigma derived by incr_lambda

def d_sigma_d_lambda (E, sigma, incr_lambda):
    d_sigma_d_lambda=np.zeros((2))
    d_sigma_d_lambda[0]= (E*(sigma[1]-3*E**2*sigma[1]*(incr_lambda**2)-2*sigma[0]*(1+3*E*incr_lamdba*(1+E*incr_lambda)))/((1+E*incr_lambda)**2*(1+3*E*incr_lambda)))
    d_sigma_d_lambda[1]= (E*(sigma[0]-3*E**2*sigma[0]*(incr_lambda**2)-2*sigma[1]*(1+3*E*incr_lamdba*(1+E*incr_lambda)))/((1+E*incr_lambda)**2*(1+3*E*incr_lambda)))
    return d_sigma_d_lambda    

In [16]:
#f derivated by incr_lamba

def f_derivated (d_sigma_d_lambda, d_f_d_sigma):
    f_derivated=d_f_d_sigma*d_sigma_d_lambda
    return f_derivated

In [18]:
#increment of lambda i

def incr_lambda_i (incr_lambda, f_derivated, f_of_incr_lambda):
    incr_lambda_i=np.zeros(2)
    incr_lambda_i[0]=incr_lambda[0]-f_of_incr_lambda/f_derivated[0]
    incr_lambda_i[1]=incr_lambda[1]-f_of_incr_lambda/f_derivated[1]
    return incr_lambda_i

In [20]:
#stress vector at the end of the iteration i

def stress_vector_i (sigma, E, incr_lambda_i):
    stress_vector_i=np.zeros((2))
    stress_vector_i[0]=(sigma[0]+E*incr_lambda_i[0]*(sigma[1]+2*sigma[0]))/((1+E*incr_lambda_i[0])**2*(1+3*E*incr_lambda_i[0]))
    stress_vector_i[1]=(sigma[1]+2*E*incr_lambda_i[1]*sigma[1]+E*incr_lambda_i[1]*sigma[0])/(1+4*E*incr_lambda_i[1]+3*E**2*incr_lambda[1]**2)
    return stress_vector_i

In [22]:
#stress trial

def sigma_trial (sigma,D, strain_incr):
    sigma_trial=sigma+D@strain_incr
    return sigma_trial

In [ ]:
sigma=sigma_0
f=f(sigma, sigma_y)
n=(1, 1) #ATTENTION !!!!!!! à modifier !!!!
incr_lambda=((D @ n.T @ n @ D) / (n @ D @ n.T)) @ strain_incr
iteration=20 #nb of iteration

#for each step i:

for i in range(iteration):

    #at step i
    sigma_trial=sigma_trial(sigma,D, strain_incr)
    f_trial=f(sigma_trial, sigma_y)

    #at step i+1
    d_f_d_sigma=d_f_d_sigma (f, sigma)
    d_sigma_d_lambda=d_sigma_d_lambda(E, sigma, incr_lambda)
    d_f_d_lambda=f_derivated(d_sigma_d_lambda, d_f_d_sigma)
    incr_lambda=incr_lambda_i (incr_lambda, f_derivated, f_of_incr_lambda)
    sigma=stress_vector_i (sigma, E, incr_lambda_i)
    f=f (sigma, sigma_y)

    if f/f_trial < 0.0001:
        print ("Convergence is reached at step ", i)
        print(f'sigma={sigma}')
        print(f'f={f}')
        break

    if i==iteration and f/f_trial>0.0001:
        return error


In [9]:
# Stress increment from strain increment

plastic_strain_incr = np.zeros((2,))
n = np.zeros((2,))

# First forward step

stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr, n, D)

stress = sigma_0 + stress_incr

print(f"Stress at first step: {stress} Mpa")

if check_failure(stress) == True:
    alpha = np.sqrt((sigma_y**2) / (stress[0]**2 + stress[1]**2 - stress[0] * stress[1]))
    stress_y = alpha * stress

elastic_strain_incr, plastic_strain_incr = strain_decomposition(strain_incr, alpha)

# Second forward step STILL NEED TO COMPUTE N

sub_increments = 2

plastic_strain_incr_2 = plastic_strain_incr / sub_increments

for sub_increment in range(sub_increments):
    n = norm_criterion_surface(stress_y)
    stress_incr = stress_incr_computation(strain_incr, plastic_strain_incr_2, n, D)
    stress = stress + stress_incr
    print(f"Stress, sub-increment {sub_increment}: {stress}")

print(f"Stress at second step: {stress} Mpa")





Stress at first step: [600. 800.] Mpa
Stress, sub-increment 0: [500. 900.]
Stress, sub-increment 1: [ 400. 1000.]
Stress at second step: [ 400. 1000.] Mpa
